## Check the setup and connect to the database

In [1]:
%run "../01-check_setup.ipynb"

SAP HANA Client for Python: 2.26.25091602
Connected to SAP HANA db version 4.00.000.00.1758012768 (fa/CE2025.28) 
at c5889dd5-e0f6-4930-8408-94d53ca61dbf.hna0.prod-us10.hanacloud.ondemand.com:443 as CODEJAMHANAAI00
Current time on the SAP HANA server: 2025-09-25 12:26:02.588000


In [2]:
myconn.get_tables(schema='NHTSA')

TABLE_NAME
0  COMPLAINTS

In [3]:
import pandas as pd

Loading data from https://www.nhtsa.gov/nhtsa-datasets-and-apis#complaints

In [4]:
import_df = pd.read_csv('https://static.nhtsa.gov/odi/ffdd/cmpl/COMPLAINTS_RECEIVED_2020-2024.zip', sep='\t', header=None)
import_df.shape

/tmp/ipykernel_13448/1342734287.py:1: DtypeWarning: Columns (27,28,29,30,33,38,41,44) have mixed types. Specify dtype option on import or set low_memory=False.
  import_df = pd.read_csv('https://static.nhtsa.gov/odi/ffdd/cmpl/COMPLAINTS_RECEIVED_2020-2024.zip', sep='\t', header=None)


(418412, 49)

In [5]:
import_df.rename(columns={
    0: 'CMPLID',
    1: 'ODINO',
    2: 'MFR_NAME',
    3: 'MAKETXT',
    4: 'MODELTXT',
    5: 'YEARTXT',
    6: 'CRASH',
    7: 'FAILDATE',
    8: 'FIRE',
    9: 'INJURED',
    10: 'DEATHS',
    11: 'COMPDESC',
    12: 'CITY',
    13: 'STATE',
    14: 'VIN',
    15: 'DATEA',
    16: 'LDATE',
    17: 'MILES',
    18: 'OCCURENCES',
    19: 'CDESCR',
    20: 'CMPL_TYPE',
    21: 'POLICE_RPT_YN',
    22: 'PURCH_DT',
    23: 'ORIG_OWNER_YN',
    24: 'ANTI_BRAKES_YN',
    25: 'CRUISE_CONT_YN',
    26: 'NUM_CYLS',
    27: 'DRIVE_TRAIN',
    28: 'FUEL_SYS',
    29: 'FUEL_TYPE',
    30: 'TRANS_TYPE',
    31: 'VEH_SPEED',
    32: 'DOT',
    33: 'TIRE_SIZE',
    34: 'LOC_OF_TIRE',
    35: 'TIRE_FAIL_TYPE',
    36: 'ORIG_EQUIP_YN',
    37: 'MANUF_DT',
    38: 'SEAT_TYPE',
    39: 'RESTRAINT_TYPE',
    40: 'DEALER_NAME',
    41: 'DEALER_TEL',
    42: 'DEALER_CITY',
    43: 'DEALER_STATE',
    44: 'DEALER_ZIP',
    45: 'PROD_TYPE',
    46: 'REPAIRED_YN',
    47: 'MEDICAL_ATTN',
    48: 'VEHICLES_TOWED_YN'
}, inplace=True)

In [6]:
# Combine them into a single DataFrame
pd.DataFrame({
    'Original Dtypes': import_df.dtypes,
    'Converted Dtypes': import_df.convert_dtypes().dtypes
})

Original Dtypes Converted Dtypes
CMPLID                      int64            Int64
ODINO                       int64            Int64
MFR_NAME                   object   string[python]
MAKETXT                    object   string[python]
MODELTXT                   object   string[python]
YEARTXT                   float64            Int64
CRASH                      object   string[python]
FAILDATE                    int64            Int64
FIRE                       object   string[python]
INJURED                     int64            Int64
DEATHS                      int64            Int64
COMPDESC                   object   string[python]
CITY                       object   string[python]
STATE                      object   string[python]
VIN                        object   string[python]
DATEA                       int64            Int64
LDATE                       int64            Int64
MILES                     float64            Int64
OCCURENCES                float64            Int64
CDESCR                     object   string[python]
CMPL_TYPE                  object   string[python]
POLICE_RPT_YN              object   string[python]
PURCH_DT                  float64            Int64
ORIG_OWNER_YN              object   string[python]
ANTI_BRAKES_YN             object   string[python]
CRUISE_CONT_YN             object   string[python]
NUM_CYLS                  float64            Int64
DRIVE_TRAIN                object   string[python]
FUEL_SYS                   object   string[python]
FUEL_TYPE                  object   string[python]
TRANS_TYPE                 object   string[python]
VEH_SPEED                 float64            Int64
DOT                        object   string[python]
TIRE_SIZE                  object   string[python]
LOC_OF_TIRE                object   string[python]
TIRE_FAIL_TYPE             object   string[python]
ORIG_EQUIP_YN              object   string[python]
MANUF_DT                  float64            Int64
SEAT_TYPE                  object   string[python]
RESTRAINT_TYPE             object   string[python]
DEALER_NAME                object   string[python]
DEALER_TEL                 object           object
DEALER_CITY                object   string[python]
DEALER_STATE               object   string[python]
DEALER_ZIP                 object           object
PROD_TYPE                  object   string[python]
REPAIRED_YN                object   string[python]
MEDICAL_ATTN               object   string[python]
VEHICLES_TOWED_YN          object   string[python]

In [7]:
import_df=import_df.convert_dtypes()

In [8]:
import_df[import_df['CDESCR'].str.len() > 2048][['CMPLID']].assign(CDESCR_LEN=import_df['CDESCR'].str.len())

CMPLID  CDESCR_LEN
261272  1894665       19513
285080  1918494        9370
298962  1932387       10096
300959  1934402       17330
330529  1963999        2196

In [9]:
pd.set_option('max_colwidth', None) 
display(
    import_df
    .head(1).T
    .style.set_properties(subset=[0], **{'text-align': 'left'})
)

In [10]:
carcomplaints_hdf = hana_ml.dataframe.create_dataframe_from_pandas(myconn,
                                           import_df[import_df['CDESCR'].str.len() <= 2048],
                                            table_name="COMPLAINTS",
                                            schema='NHTSA',
                                            force=True)

100%|██████████| 9/9 [00:19<00:00,  2.14s/it]


In [11]:
carcomplaints_hdf.shape

[418382, 49]

In [12]:
myconn.get_tables(schema="NHTSA")

TABLE_NAME
0  COMPLAINTS

In [13]:
myconn.table("COMPLAINTS", schema='NHTSA').filter("LEFT(LDATE,4) = '2024'").describe().collect().sort_values(by='unique', ascending=True)

column  count  unique  nulls          mean           std  \
9          OCCURENCES      0       0  97855           NaN           NaN   
34          TIRE_SIZE      0       0  97855           NaN           NaN   
26      ORIG_OWNER_YN  96425       2   1430           NaN           NaN   
25      POLICE_RPT_YN  97855       2      0           NaN           NaN   
28     CRUISE_CONT_YN  96425       2   1430           NaN           NaN   
30           FUEL_SYS     47       2  97808           NaN           NaN   
18               FIRE  97855       2      0           NaN           NaN   
27     ANTI_BRAKES_YN  96425       2   1430           NaN           NaN   
24          CMPL_TYPE  97855       2      0           NaN           NaN   
37      ORIG_EQUIP_YN    503       2  97352           NaN           NaN   
48  VEHICLES_TOWED_YN  96425       2   1430           NaN           NaN   
47       MEDICAL_ATTN  97855       2      0           NaN           NaN   
46        REPAIRED_YN    631       2  97224           NaN           NaN   
17              CRASH  97855       2      0           NaN           NaN   
32         TRANS_TYPE     55       2  97800           NaN           NaN   
29        DRIVE_TRAIN    539       4  97316           NaN           NaN   
45          PROD_TYPE  97855       4      0           NaN           NaN   
38          SEAT_TYPE    249       4  97606           NaN           NaN   
31          FUEL_TYPE    535       4  97320           NaN           NaN   
5              DEATHS  97855       5      0  1.737264e-03      0.318037   
11           NUM_CYLS    499       5  97356  5.124248e+00      1.366771   
35        LOC_OF_TIRE    536       5  97319           NaN           NaN   
39     RESTRAINT_TYPE    206       5  97649           NaN           NaN   
36     TIRE_FAIL_TYPE     78       6  97777           NaN           NaN   
4             INJURED  97855      11      0  3.119922e-02      0.398401   
10           PURCH_DT    288      30  97567  2.020961e+07  31355.635502   
2             YEARTXT  97855      40      0  2.142011e+03    986.943207   
43       DEALER_STATE  11247      56  86608           NaN           NaN   
21              STATE  97855      62      0           NaN           NaN   
12          VEH_SPEED  46688     118  51167  3.492388e+01     30.569538   
13           MANUF_DT    202     136  97653  2.021970e+07  15078.771207   
14           MFR_NAME  97855     262      0           NaN           NaN   
33                DOT    434     271  97421           NaN           NaN   
15            MAKETXT  97855     306      0           NaN           NaN   
6               DATEA  97855     366      0  2.024066e+07    345.492048   
7               LDATE  97855     366      0  2.024066e+07    345.492048   
19           COMPDESC  97855     459      0           NaN           NaN   
16           MODELTXT  97855    1653      0           NaN           NaN   
3            FAILDATE  97855    2055      0  2.023722e+07  11585.137252   
41         DEALER_TEL   4397    2893  93458           NaN           NaN   
8               MILES  17926    3044  79929  7.989531e+04  58742.550799   
42        DEALER_CITY  11323    3435  86532           NaN           NaN   
44         DEALER_ZIP  10018    3503  87837           NaN           NaN   
40        DEALER_NAME  11782    7225  86073           NaN           NaN   
20               CITY  97853   15484      2           NaN           NaN   
22                VIN  95456   41974   2399           NaN           NaN   
23             CDESCR  97855   66285      0           NaN           NaN   
1               ODINO  97855   67690      0  1.159814e+07  20830.536141   
0              CMPLID  97855   97855      0  2.002947e+06  28253.240315   

           min         max      median  25_percent_cont  25_percent_disc  \
9          NaN         NaN         NaN              NaN              NaN   
34         NaN         NaN         NaN              NaN              NaN   
26         NaN         NaN         N